Wave Equation
===

We solve the first order wave equation by a matrix-free explicit DG method.

\begin{eqnarray*}
\frac{\partial p}{\partial t} & = & \operatorname{div} u \\
\frac{\partial u}{\partial t} & = & \nabla p
\end{eqnarray*}

We obtain the ODE 
\begin{eqnarray*}
M_p \dot{p} & = & -B^T u \\
M_u \dot{u} & = & B p
\end{eqnarray*}

form a simple DG version with central fluxes. The discrete gradient $B$ is defined by the bilinear-form
$$
b(p,v) = \sum_{T}
\Big\{ \int_T \nabla p  \, v + \int_{\partial T} (\{ p \} - p) \, v_n \, ds \Big\} 
$$

It conserves energy, but is not free of spurious modes.

Hesthaven+Warbuton: Nodal Discontinuous Galerkin Methods

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw

dim = 2

if dim==2:
    from netgen.geom2d import SplineGeometry
    geo = SplineGeometry()
    geo.AddRectangle( (-1, -1), (1, 1), bcs = ("bottom", "right", "top", "left"))
    geo.AddCircle ( (0.5, 0), r=0.2, leftdomain=0, rightdomain=1)
    h = 0.02

else:
    from netgen.csg import *
    geo = CSGeometry()
    box = OrthoBrick (Pnt(-1,-1,-1), Pnt(1,1,0))
    hole = Sphere( Pnt(0.5, 0, 0), 0.2 )
    geo.Add ( (box-hole).bc("outer"))
    h = 0.1
    
    
mesh = Mesh( geo.GenerateMesh(maxh=h))
mesh.Curve(3)
Draw(mesh);

A new component is the `TraceOperator`:

Space provide geometry-free linear operators mapping form the element space to the face space. Face values can be averaged, or are summed up.

In [ ]:
order = 5
fes_p = L2(mesh, order=order+1, all_dofs_together=True)
fes_u = VectorL2(mesh, order=order, piola=True)
fes_tr = FacetFESpace(mesh, order=order+1)

traceop = fes_p.TraceOperator(fes_tr, average=True) 

gfu = GridFunction(fes_u)
gfp = GridFunction(fes_p)
gftr = GridFunction(fes_tr)

gfp.Interpolate( exp(-400*(x**2+y**2+z**2)))
gftr.vec.data = traceop * gfp.vec

In [ ]:
if dim == 2:
    Draw (gfp, order=3)
else:
    gftr.vec.data = traceop * gfp.vec
    Draw (gftr, draw_vol=False, order=3);

In [ ]:
p = fes_p.TrialFunction()
v = fes_u.TestFunction()
phat = fes_tr.TrialFunction()

n = specialcf.normal(mesh.dim)

We define bilinear-forms for the element-wise $p$, and for the facet-wise $p_{trace}$, the test-function is $v$. Thanks to the co-variant mapping of $v$, both forms are independent of element-geometry, and only one element matrix is calculated for the reference element(s): 

In [ ]:
Bel = BilinearForm(trialspace=fes_p, testspace=fes_u, geom_free = True)
Bel += grad(p)*v * dx -p*(v*n) * dx(element_boundary=True)
Bel.Assemble()

Btr = BilinearForm(trialspace=fes_tr, testspace=fes_u, geom_free = True)
Btr += phat * (v*n) *dx(element_boundary=True)
Btr.Assemble();

Combine linear operators:

In [ ]:
B = Bel.mat + Btr.mat @ traceop

Inverse mass matrices: either (block)diagonal, or operator application via sum factorization:

In [ ]:
invmassp = fes_p.Mass(1).Inverse()
invmassu = fes_u.Mass(1).Inverse()

In [ ]:
gfp.Interpolate( exp(-400*(x**2+y**2+z**2)))
gfu.vec[:] = 0

if dim == 2:
    scene = Draw (gfp, order=3);
else:
    scene = Draw (gftr, draw_vol=False, order=3);

t = 0
tend = 20
dt = 0.5 * h / (order+1)**2
print ("dt = ", dt)

cnt = 0
with TaskManager(pajetrace=10**8):
    while t < tend:
        t = t+dt
        gfu.vec.data += dt * invmassu @ B * gfp.vec
        gfp.vec.data -= dt * invmassp @ B.T * gfu.vec
        cnt = cnt+1
        if cnt%10 == 0:
            if dim == 3:
                gftr.vec.data = traceop * gfp.vec
            scene.Redraw()